# Funciones Auxiliares

In [ ]:
import pathlib
import os
from PIL import Image
import numpy as np
import cv2
import tensorflow as tf
import random
import argparse
from tensorflow.python.client import device_lib
from __future__ import print_function

class data_utils:
    def getpaths(path):
        """
        Get all image paths from folder 'path' while avoiding ._ files.
        """
        im_paths = []
        for fil in os.listdir(path):
              if '.png' in fil:
                if "._" in fil:
                      #avoid dot underscore
                    pass
                else:
                      im_paths.append(os.path.join(path, fil))
      return im_paths

    def make_dataset(paths, scale, mean):
      """
      Python generator-style dataset. Creates 48x48 low-res and corresponding high-res patches.
      """
      size_lr = 48
      size_hr = size_lr * scale

      for p in paths:
          # normalize
          im_norm = cv2.imread(p.decode(), 3).astype(np.float32) - mean

          # random flip
          r = random.randint(-1, 2)
          if not r == 2:
              im_norm = cv2.flip(im_norm, r)

          # divisible by scale - create low-res
          hr = im_norm[0:(im_norm.shape[0] - (im_norm.shape[0] % scale)),
                    0:(im_norm.shape[1] - (im_norm.shape[1] % scale)), :]
          lr = cv2.resize(hr, (int(hr.shape[1] / scale), int(hr.shape[0] / scale)),
                          interpolation=cv2.INTER_CUBIC)

          numx = int(lr.shape[0] / size_lr)
          numy = int(lr.shape[1] / size_lr)

          for i in range(0, numx):
              startx = i * size_lr
              endx = (i * size_lr) + size_lr

              startx_hr = i * size_hr
              endx_hr = (i * size_hr) + size_hr

              for j in range(0, numy):
                  starty = j * size_lr
                  endy = (j * size_lr) + size_lr
                  starty_hr = j * size_hr
                  endy_hr = (j * size_hr) + size_hr

                  crop_lr = lr[startx:endx, starty:endy]
                  crop_hr = hr[startx_hr:endx_hr, starty_hr:endy_hr]

                  x = crop_lr.reshape((size_lr, size_lr, 3))
                  y = crop_hr.reshape((size_hr, size_hr, 3))

                  yield x, y

  def calcmean(imageFolder, bgr):
      """
      Calculates the mean of a dataset.
      """
      paths = getpaths(imageFolder)

      total_mean = [0, 0, 0]
      im_counter = 0

      for p in paths:

          image = np.asarray(Image.open(p))

          mean_rgb = np.mean(image, axis=(0, 1), dtype=np.float64)

          if im_counter % 1000 == 0:
              print("Total mean: {} | current mean: {}".format(total_mean, mean_rgb))

          total_mean += mean_rgb
          im_counter += 1

      total_mean /= im_counter

      # rgb to bgr
      if bgr is True:
          total_mean = total_mean[...,::-1]

      return total_mean

# Parámetros

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' #gets rid of avx/fma warning

B=32
F=256
scale=2
batch=16
epochs=5
lr=0.0001
load_flag= False
mean = [30.7314651342487, 34.951122349740935, 35.78383743336787]
ckpt_path = "/content/drive/MyDrive/CKPT_dirLAST/x2/"
imagefolder="/content/drive/MyDrive/TheLastofUs/HD1"
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True

# Entrenamiento

In [ ]:
tf.compat.v1.disable_eager_execution()
def train(imagefolder):
        # Create training dataset
        train_image_paths = data_utils.getpaths(imagefolder)
        train_dataset = tf.data.Dataset.from_generator(generator=data_utils.make_dataset,
                                                 output_types=(tf.float32, tf.float32),
                                                 output_shapes=(tf.TensorShape([None, None, 3]), 
                                                                tf.TensorShape([None, None, 3])),
                                                 args=[train_image_paths, scale, mean])
        train_dataset = train_dataset.padded_batch(batch, padded_shapes=([None, None, 3],[None, None, 3]))

        # Make the iterator and its initializers
        output_types=tf.compat.v1.data.get_output_types(train_dataset)
        output_shapes=tf.compat.v1.data.get_output_shapes(train_dataset)
        train_val_iterator = tf.compat.v1.data.Iterator.from_structure(output_types, output_shapes)
        train_initializer = train_val_iterator.make_initializer(train_dataset)

        handle = tf.compat.v1.placeholder(tf.string, shape=[])
        iterator = tf.compat.v1.data.Iterator.from_string_handle(handle, output_types, output_shapes)
        LR, HR = iterator.get_next()

        # Edsr model
        print("\nRunning EDSR.")
        edsrObj = Edsr(B, F, scale)
        out, loss, train_op, psnr, ssim, lr1 = edsrObj.model(x=LR, y=HR, lr=lr)

        # -- Training session
        with tf.compat.v1.Session(config=config) as sess:

            train_writer = tf.compat.v1.summary.FileWriter('/content/drive/MyDrive/logsLAST/train', sess.graph)
            sess.run(tf.compat.v1.global_variables_initializer())

            saver = tf.compat.v1.train.Saver()

            # Create check points directory if not existed, and load previous model if specified.
            if not os.path.exists(ckpt_path):
                os.makedirs(ckpt_path)
            else:
                if os.path.isfile(ckpt_path + "edsr_ckpt" + ".meta"):
                    if load_flag:
                        saver.restore(sess, tf.train.latest_checkpoint(ckpt_path))
                        print("\nLoaded checkpoint.")
                    if not load_flag:
                        print("No checkpoint loaded. Training from scratch.")

            global_step = 0
            tf.convert_to_tensor(global_step)

            train_val_handle = sess.run(train_val_iterator.string_handle())

            print("Training...")
            for e in range(1, epochs+1):

                sess.run(train_initializer)
                step, train_loss = 0, 0

                try:
                    while True:
                        o, l, t, l_rate = sess.run([out, loss, train_op, lr1], feed_dict={handle:train_val_handle,
                                                                                         edsrObj.global_step: global_step})
                        train_loss += l
                        step += 1
                        global_step += 1
                        #print(global_step)

                        if step % 1000 == 0:
                            save_path = saver.save(sess, ckpt_path + "edsr_ckpt")
                            print("Step nr: [{}/{}] - Loss: {:.5f} - Lr: {:.7f}".format(step, "?", 
                                                                                        float(train_loss/step), l_rate))

                except tf.errors.OutOfRangeError:
                    pass

                save_path = saver.save(sess, ckpt_path + "edsr_ckpt")

            print("Training finished.")
            train_writer.close()

# Estructura EDSR

In [ ]:
class Edsr:

    def __init__(self, B, F, scale):
        self.B = B
        self.F = F
        self.scale = scale
        self.global_step = tf.compat.v1.placeholder(tf.int32, shape=[], name="global_step")
        self.scaling_factor = 0.1
        self.bias_initializer = tf.constant_initializer(value=0.0)
        self.PS = 3 * (scale*scale) #channels x scale^2
        self.xavier = tf.initializers.GlorotUniform()

        # -- Filters & Biases --
        self.resFilters = list()
        self.resBiases = list()

        for i in range(0, B*2):
            self.resFilters.append(tf.compat.v1.get_variable("resFilter%d" % (i), shape=[3,3,F,F], initializer=self.xavier))
            self.resBiases.append(tf.compat.v1.get_variable(name="resBias%d" % (i), shape=[F], initializer=self.bias_initializer))

        self.filter_one = tf.compat.v1.get_variable("resFilter_one", shape=[3,3,3,F], initializer=self.xavier)
        self.filter_two = tf.compat.v1.get_variable("resFilter_two", shape=[3,3,F,F], initializer=self.xavier)
        self.filter_three = tf.compat.v1.get_variable("resFilter_three", shape=[3,3,F,self.PS], initializer=self.xavier)

        self.bias_one = tf.compat.v1.get_variable(shape=[F], initializer=self.bias_initializer, name="BiasOne")
        self.bias_two = tf.compat.v1.get_variable(shape=[F], initializer=self.bias_initializer, name="BiasTwo")
        self.bias_three = tf.compat.v1.get_variable(shape=[self.PS], initializer=self.bias_initializer, name="BiasThree")


    def model(self, x, y, lr):
        """
        Implementation of EDSR: https://arxiv.org/abs/1707.02921.
        """

        # -- Model architecture --

        # first conv
        x = tf.nn.conv2d(x, filters=self.filter_one, strides=[1, 1, 1, 1], padding='SAME')
        x = x + self.bias_one
        out1 = tf.identity(x)

        # all residual blocks
        for i in range(self.B):
            x = self.resBlock(x, (i*2))

        # last conv
        x = tf.nn.conv2d(x, filters=self.filter_two, strides=[1, 1, 1, 1], padding='SAME')
        x = x + self.bias_two
        x = x + out1

        # upsample via sub-pixel, equivalent to depth to space
        x = tf.nn.conv2d(x, filters=self.filter_three, strides=[1, 1, 1, 1], padding='SAME')
        x = x + self.bias_three
        out = tf.nn.depth_to_space(x, self.scale, data_format='NHWC', name="NHWC_output")
        
        # -- --

        # some outputs
        out_nchw = tf.transpose(out, [0, 3, 1, 2], name="NCHW_output")
        psnr = tf.image.psnr(out, y, max_val=255.0)
        loss = tf.compat.v1.losses.absolute_difference(out, y) #L1
        ssim = tf.image.ssim(out, y, max_val=255.0)
        
        # (decaying) learning rate
        lr = tf.compat.v1.train.exponential_decay(lr,
                                        self.global_step,
                                        decay_steps=3000,
                                        decay_rate=0.95,
                                        staircase=True)
        # gradient clipping
        optimizer = tf.compat.v1.train.AdamOptimizer(lr)
        gradients, variables = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        train_op = optimizer.apply_gradients(zip(gradients, variables))

        return out, loss, train_op, psnr, ssim, lr

    def resBlock(self, inpt, f_nr):
        x = tf.nn.conv2d(inpt, filters=self.resFilters[f_nr], strides=[1, 1, 1, 1], padding='SAME')
        x = x + self.resBiases[f_nr]
        x = tf.nn.relu(x)

        x = tf.nn.conv2d(x, filters=self.resFilters[f_nr+1], strides=[1, 1, 1, 1], padding='SAME')
        x = x + self.resBiases[f_nr+1]
        x = x * self.scaling_factor

        return inpt + x